In [1]:
from tensorflow.keras.models import load_model

import cv2
import numpy as np

In [2]:
ageModel = load_model('ageModel.h5')
genderModel = load_model('genderModel.h5')

ageModel.summary()
genderModel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 50, 50, 128)       3328      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 25, 25, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 25, 25, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 256)       819456    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 256)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 512)       3

In [3]:
haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [4]:
imgSize = 50

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    cap.release()

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1600)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 900)

while True:
    
    ret, frame = cap.read()
    
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = haar.detectMultiScale(img, 5, 5)
    
    try:

        for (x, y, w, h) in faces:

            roi = img[x:x + w, y:y + h]
            roi = cv2.resize(roi, (imgSize, imgSize))
            roi = roi / 255.0
            roi = roi.reshape(1, imgSize, imgSize, 1)

            agePredict = ageModel.predict(roi)
            genderPredict = genderModel.predict(roi)

            age = np.argmax(agePredict)
            gender = np.argmax(genderPredict)

            if gender == 0:
                gender = 'Male'
            else:
                gender = 'Female'

            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(frame, 
                        '{}, {}'.format(age, gender), 
                        (x, y), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        1, 
                        (255, 0, 0), 
                        2)
    except:
        pass
    
    cv2.imshow('video', frame)
    
    k = cv2.waitKey(1) & 0xFF
    
    if k == 27:
        break

cv2.destroyAllWindows()
cap.release()